# Copernicus: query and ingest

https://documentation.dataspace.copernicus.eu/APIs/STAC.html

https://documentation.dataspace.copernicus.eu/APIs/S3.html#example-script-to-download-product-using-boto3

In [1]:
import os
os.environ["abcli_path_bash"] = "{}/git/awesome-bash-cli/bash".format(os.getenv("HOME"))

In [2]:
from blue_options import fullname, string

from blue_objects import metadata

from blue_geo import NAME, VERSION
from blue_geo.catalog.copernicus import CopernicusSentinel2Datacube, CopernicusCatalog
from blue_geo.catalog import get_datacube
from blue_geo.logger import logger

logger.info(f"{NAME}-{VERSION}.{fullname()}, built on {string.pretty_date()}")

🌐  blue_geo-4.723.1.blue_options-4.148.1, built on 03 November 2024, 19:39:19


---

# catalog

In [3]:
catalog = CopernicusCatalog()
catalog.get_list_of_collections()

['CCM',
 'COP-DEM',
 'ENVISAT',
 'GLOBAL-MOSAICS',
 'LANDSAT-5',
 'LANDSAT-7',
 'LANDSAT-8-ESA',
 'S2GLC',
 'SENTINEL-1',
 'SENTINEL-1-RTC',
 'SENTINEL-2',
 'SENTINEL-3',
 'SENTINEL-5P',
 'SENTINEL-6',
 'SMOS',
 'TERRAAQUA']

## query

In [4]:
object_name = f"query-{string.timestamp()}"
logger.info(f"📂 {object_name}")

🌐  📂 query-2024-11-03-19-39-20-64198


In [5]:
assert CopernicusSentinel2Datacube.query(
    object_name=object_name,
    bbox=[-122.78 - 0.1, 51.83 - 0.1, -122.78 + 0.1, 51.83 + 0.1],
    datetime="2024-07-30/2024-08-09",
    count=-1,
)

🌐  🔎 CopernicusSentinel2Datacube.query -> query-2024-11-03-19-39-20-64198
🌐  🔎 collections: ['SENTINEL-2']
🌐  🔎 bbox: [-122.88, 51.73, -122.68, 51.93]
🌐  🔎 datetime: 2024-07-30/2024-08-09
🌐  🔎 sortby: [{'field': 'properties.datetime', 'direction': 'desc'}]
/Users/kamangir/git/.venv/lib/python3.9/site-packages/pystac_client/item_search.py:573: DoesNotConformTo: Server does not conform to SORT
  warnings.warn(DoesNotConformTo("SORT"))
🌐  18 datacubes(s) found.
🌐  🧊 01: datacube-copernicus-sentinel_2-S2A_MSIL2A_20240731T191911_N0511_R099_T10UDC_20240801T023153-SAFE
🌐  🧊 02: datacube-copernicus-sentinel_2-S2A_MSIL2A_20240731T191911_N0511_R099_T10UEC_20240801T023153-SAFE
🌐  🧊 03: datacube-copernicus-sentinel_2-S2A_MSIL1C_20240731T191911_N0511_R099_T10UEC_20240801T003519-SAFE
🌐  🧊 04: datacube-copernicus-sentinel_2-S2A_MSIL1C_20240731T191911_N0511_R099_T10UDC_20240801T003519-SAFE
🌐  🧊 05: datacube-copernicus-sentinel_2-S2B_MSIL2A_20240802T190919_N0511_R056_T10UEC_20240802T231522-SAFE
🌐  🧊 06

In [6]:
datacube_id = metadata.get_from_object(object_name, "datacube_id")[0]
logger.info(f"🧊 {datacube_id}")

🌐  🧊 datacube-copernicus-sentinel_2-S2A_MSIL2A_20240731T191911_N0511_R099_T10UDC_20240801T023153-SAFE


## ingest

In [7]:
datacube = get_datacube(datacube_id)
assert isinstance(datacube, CopernicusSentinel2Datacube)

datacube.metadata["Item"]

<Item id=S2A_MSIL2A_20240731T191911_N0511_R099_T10UDC_20240801T023153.SAFE>

In [8]:
list_of_files = datacube.list_of_files()

logger.info("{} file(s)".format(len(list_of_files)))
for index, filename in enumerate(list_of_files):
    logger.info(f"#{index:03d} - {filename}")


🌐  95 file(s)
🌐  #000 - DATASTRIP/DS_2APS_20240801T023153_S20240731T192211/MTD_DS.xml
🌐  #001 - DATASTRIP/DS_2APS_20240801T023153_S20240731T192211/QI_DATA/FORMAT_CORRECTNESS.xml
🌐  #002 - DATASTRIP/DS_2APS_20240801T023153_S20240731T192211/QI_DATA/GENERAL_QUALITY.xml
🌐  #003 - DATASTRIP/DS_2APS_20240801T023153_S20240731T192211/QI_DATA/GEOMETRIC_QUALITY.xml
🌐  #004 - DATASTRIP/DS_2APS_20240801T023153_S20240731T192211/QI_DATA/RADIOMETRIC_QUALITY.xml
🌐  #005 - DATASTRIP/DS_2APS_20240801T023153_S20240731T192211/QI_DATA/SENSOR_QUALITY.xml
🌐  #006 - GRANULE/L2A_T10UDC_A047572_20240731T192211/AUX_DATA/AUX_CAMSFO
🌐  #007 - GRANULE/L2A_T10UDC_A047572_20240731T192211/AUX_DATA/AUX_ECMWFT
🌐  #008 - GRANULE/L2A_T10UDC_A047572_20240731T192211/IMG_DATA/R10m/T10UDC_20240731T191911_AOT_10m.jp2
🌐  #009 - GRANULE/L2A_T10UDC_A047572_20240731T192211/IMG_DATA/R10m/T10UDC_20240731T191911_B02_10m.jp2
🌐  #010 - GRANULE/L2A_T10UDC_A047572_20240731T192211/IMG_DATA/R10m/T10UDC_20240731T191911_B03_10m.jp2
🌐  #011 -

In [9]:
success, _ = datacube.ingest()
assert success

🌐  blue_geo.catalog.generic.generic.classes.CopernicusSentinel2Datacube.ingest(): metadata @ datacube-copernicus-sentinel_2-S2A_MSIL2A_20240731T191911_N0511_R099_T10UDC_20240801T023153-SAFE
🌐  ingesting 26 file(s)...
100%|██████████| 26/26 [00:27<00:00,  1.07s/it]


---

In [10]:
# END